In [20]:
from kafka import KafkaProducer
import json
import uuid
from datetime import datetime
import time
import random

producer = KafkaProducer(
    bootstrap_servers=f"kafka-bigspoke-padmashreemp-6a22.aivencloud.com:18221",
    security_protocol="SSL",
    ssl_cafile="ca.pem",
    ssl_certfile="service.cert",
    ssl_keyfile="service.key",
)

numOfCompartments = 2

# Generate random temperature value
def generate_temperature():
    min_temp = -25  # Minimum temperature value
    max_temp = 10  # Maximum temperature value
    temperature = round(random.uniform(min_temp, max_temp), 2)
    return temperature

# Generate random air pressure value
def generate_air_pressure():
    min_pressure = 50  # Minimum air pressure value in kPa
    max_pressure = 100  # Maximum air pressure value in kPa
    air_pressure = round(random.uniform(min_pressure, max_pressure), 2)
    return air_pressure

# Generate random speed value
def generate_speed():
    min_speed = 0  # Minimum speed value in km/h
    max_speed = 200  # Maximum speed value in km/h
    speed = round(random.uniform(min_speed, max_speed), 2)
    return speed

# Generate random brake status
def generate_brake_status():
    brake_status = random.choice(['engaged', 'released'])
    return brake_status

# Generate random Latitude and Longitude
def generate_random_coordinates():
    base_latitude = 37.7749  
    base_longitude = -122.4194  
    max_variation = 0.01  
    latitude_variation = random.uniform(-max_variation, max_variation)
    longitude_variation = random.uniform(-max_variation, max_variation)

    # Apply the variations to the base coordinates
    latitude = round(base_latitude + latitude_variation, 6)
    longitude = round(base_longitude + longitude_variation, 6)

    return latitude, longitude

# Generate random event data
def generate_event(sensor_type, compartment_id):
    event_id = str(uuid.uuid4())
    timestamp = datetime.now().isoformat()
    latitude, longitude = generate_random_coordinates()

    payload = {
        'event_id': event_id,
        'compartment_id': compartment_id,
        'sensor_type': sensor_type,
        'time_stamp': timestamp,
        'latitude': latitude,
        'longitude': longitude
    }

    if sensor_type == 'temperature':
        sensor_id = 'ts' + str(compartment_id)
        sensor_name = 'temperature_sensor_' + str(compartment_id)
        temperature = generate_temperature() # Generate temperature-specific data
        payload['temperature'] = temperature

    elif sensor_type == 'air_pressure':
        sensor_id = 'ps'+ str(compartment_id)
        sensor_name = 'air_pressure_sensor_' + str(compartment_id)
        air_pressure = generate_air_pressure() # Generate air pressure-specific data
        payload['air_pressure'] = air_pressure
        
    else:
        sensor_id = 'sb1'
        sensor_name = 'brake_speed_sensor_1'
        speed = generate_speed()         # Generate speed and brake-specific data
        brake_status = generate_brake_status() 
        payload['speed'] = speed
        payload['brake_status'] = brake_status

    payload['sensor_id'] = sensor_id
    payload['sensor_name'] = sensor_name
        
    return json.dumps(payload)

for i in range(5):
    # Generate temperature and air pressure event for each Compartment
    for compartmentID in range(numOfCompartments):
        TOPIC_NAME = "temperature_topic"
        temperature_event = generate_event('temperature', compartmentID)
        producer.send(TOPIC_NAME, temperature_event.encode('utf-8'))
        
        TOPIC_NAME = "air_pressure_topic"
        air_pressure_event = generate_event('air_pressure', compartmentID)
        producer.send(TOPIC_NAME, air_pressure_event.encode('utf-8'))
    
    # Generate speed and brake event 
    TOPIC_NAME = "speed_brake_topic"
    brake_speed_event = generate_event('speed_brake', -1)
    producer.send(TOPIC_NAME, brake_speed_event.encode('utf-8'))
    
    print(f"Message sent: {message}")
    time.sleep(1)

print(f"Complerted.")
producer.close()

Message sent: Hello from Python using SSL 2!
Message sent: Hello from Python using SSL 2!
Message sent: Hello from Python using SSL 2!
Message sent: Hello from Python using SSL 2!
Message sent: Hello from Python using SSL 2!
Complerted.
